# Linear Cryptanalysis

In [1]:
from classes import *
from utils import *
from data import known_ciphertext, known_plaintext
from analysis import *

- The implementation of the Substitution Permutation Network can be found in classes.py.  
- The functions used for the linear cryptanalysis approach can be found in analysis.py.
- The known plaintext-ciphertext pairs used in the Known Plaintext Attack can be found in data.py.
- Other necessary functions can be found in utils.py.

## 1 Analysing Cipher Components
We create the linear approximation table and initialise a Substitution Box.

In [2]:
linear_approx_table = {} # save as dictionary before visualisation
sbox = SubBox()

We create linear equations and calculate the probabilities for each equation holding true. For example, take the equation as follows: 

$\quad$ X<sub>2</sub> $\oplus$ X<sub>3</sub> $\oplus$ X<sub>4</sub> = Y<sub>2</sub>

This means we apply a binary mask of [0 1 1 1] to the input bits and a binary mask of [0 1 0 0] to the corresponding output bits, and check to see if the XOR sum of the left hand side of the equation is equal to the XOR sum of the right hand side of the equation.

Each pair of input mask and output mask gives a unique equation. For every input-output pair (16 pairs, where each input gives 1 output), we count the number of times the equation is true.

For a perfectly random substitution box, the distribution should be $\frac{1}{2}$ for all equations. As practically the randomness is not perfect, there is a bias for each equation. The bias refers to the deviation from $\frac{1}{2}$ that the equation holds true.

To calculate the deviation, we take the number of times the equation is true and subtract it by half the number of times it is tested, which in this case is 8. The table is then populated by the deviations. 

In [3]:
linear_approx_table = generate_linear_approx_table(linear_approx_table, sbox)

### 1.1 Representing the table as a Pandas DataFrame

In [4]:
visualise_linear_approx_table(linear_approx_table)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,-2,-2,4,0,2,-2,0,0,2,-2,0,4,2,2,0
2,0,4,0,0,0,4,0,0,-2,2,2,2,2,-2,-2,-2
3,0,-2,2,-4,0,2,2,0,2,4,0,-2,2,0,0,2
4,0,2,2,0,0,2,2,0,2,-4,0,-2,2,4,0,-2
5,0,4,0,0,0,0,0,-4,2,2,-2,2,-2,2,2,2
6,0,2,-2,0,4,-2,2,4,0,2,-2,0,0,2,-2,0
7,0,0,0,0,4,0,-4,0,4,0,4,0,0,0,0,0
8,0,0,-2,-2,-2,2,0,4,0,0,2,2,-2,2,4,0
9,0,-2,4,2,2,0,2,0,0,2,0,2,-2,0,2,-4


The rows of the table correspond to the binary mask of the input, whereas the columns correspond to the binary mask of the output. For example, the equation X<sub>2</sub> $\oplus$ X<sub>3</sub> $\oplus$ X<sub>4</sub> = Y<sub>2</sub> corresponds to row 7 and column 4, which has a bias of +$\frac{4}{16}$.

## 2 Constructing Linear Approximations for the Cryptographic System

### 2.1 Subkey representing the 8 most significant bits

We will first attempt to derive the linear equations for the target partial subkey that represents the 8 most significant bits of the last round key.

We let U<sub>i</sub> and V<sub>i</sub> represent the 16-bit block of bits at the input and output of round i's substitution boxes, respectively. U<sub>i,j</sub> and V<sub>i,j</sub> will represent the j-th bit at the i-th round, where bits are numbered from 1 to 16 from left to right. We also let K<sub>i,j</sub> represent the j-th bit of the subkey at round i that will be used in the XOR operation with U<sub>i,j</sub> to produce V<sub>i,j</sub>.  

To represent the substition boxes, S<sub>ij</sub> denotes the j-th substition box for round i, where the boxes are numbered from 1 to 4 from left to right.

#### We use the following approximations of the S-box:  
$\quad$ S<sub>11</sub>: X<sub>2</sub> $\oplus$ X<sub>3</sub> $\oplus$ X<sub>4</sub> = Y<sub>2</sub> 

$\quad$ S<sub>24</sub>: X<sub>4</sub> = Y<sub>3</sub> $\oplus$ Y<sub>4</sub>  

$\quad$ S<sub>31</sub>: X<sub>2</sub> = Y<sub>1</sub> $\oplus$ Y<sub>4</sub>  

$\quad$ S<sub>34</sub>: X<sub>2</sub> = Y<sub>1</sub> $\oplus$ Y<sub>4</sub>  

#### Equation 1:
$\quad$ V<sub>1,2</sub> = U<sub>1,2</sub> $\oplus$ U<sub>1,3</sub> $\oplus$ U<sub>1,4</sub>
= (P<sub>2</sub> $\oplus$ K<sub>1,2</sub>) $\oplus$ (P<sub>3</sub> $\oplus$ K<sub>1,3</sub>) $\oplus$ (P<sub>4</sub> $\oplus$ K<sub>1,4</sub>)

This equation has a probability of $\frac{12}{16}$ with bias $+\frac{1}{4}$.

#### Equation 2:
$\quad$ V<sub>2,15</sub> $\oplus$ V<sub>2,16</sub>= U<sub>2,16</sub>
= K<sub>2,16</sub> $\oplus$ V<sub>1,2</sub>

This equation has a probability of $\frac{12}{16}$ with bias $+\frac{1}{4}$.

#### Equation 3:
Move everything to LHS for equation 2.  
  
$\quad$ V<sub>2,15</sub> $\oplus$ V<sub>2,16</sub> $\oplus$ K<sub>2,16</sub> $\oplus$ V<sub>1,2</sub> = 0

Combining equations 1 and 2.  
  
$\quad$ V<sub>2,15</sub> $\oplus$ V<sub>2,16</sub> $\oplus$ K<sub>2,16</sub> $\oplus$ (P<sub>2</sub> $\oplus$ K<sub>1,2</sub>) $\oplus$ (P<sub>3</sub> $\oplus$ K<sub>1,3</sub>) $\oplus$ (P<sub>4</sub> $\oplus$ K<sub>1,4</sub>) = 0
  
Calculating probabilty.  
  
$\quad$ $\frac{1}{2} + 2(\frac{3}{4}-\frac{1}{2})(\frac{3}{4}-\frac{1}{2}) = \frac{5}{8}$
  
This equation has a probability of $\frac{5}{8}$ with bias $+\frac{1}{8}$.

#### Equation 4:
$\quad$ $(a)$ V<sub>3,1</sub> $\oplus$ V<sub>3,4</sub> = U<sub>3,2</sub> = V<sub>2,16</sub> $\oplus$ K<sub>3,2</sub>  
  
$\quad$ $(b)$ V<sub>3,13</sub> $\oplus$ V<sub>3,16</sub> = U<sub>3,14</sub> = V<sub>2,15</sub> $\oplus$ K<sub>3,14</sub>   
  
Move everything to LHS for equation 4 $(a)$.
  
$\quad$ V<sub>3,1</sub> $\oplus$ V<sub>3,4</sub> $\oplus$ V<sub>2,16</sub> $\oplus$ K<sub>3,2</sub> = 0  
  
Move everything to LHS for equation 4 $(b)$.  

$\quad$ V<sub>3,13</sub> $\oplus$ V<sub>3,16</sub> $\oplus$ V<sub>2,15</sub> $\oplus$ K<sub>3,14</sub> = 0  
  
Combining the equations.  

$\quad$ V<sub>3,1</sub> $\oplus$ V<sub>3,4</sub> $\oplus$ V<sub>2,16</sub> $\oplus$ K<sub>3,2</sub> $\oplus$ V<sub>3,13</sub> $\oplus$ V<sub>3,16</sub> $\oplus$ V<sub>2,15</sub> $\oplus$ K<sub>3,14</sub> = 0

Calculating probabilty.  

$\quad$ $\frac{1}{2} + 2(\frac{1}{4}-\frac{1}{2})(\frac{1}{4}-\frac{1}{2}) = \frac{5}{8}$  
  
This equation has a probability of $\frac{5}{8}$ with bias $+\frac{1}{8}$.

#### Equation 5:
Combining equations 3 and 4.  

$\quad$ V<sub>2,15</sub> $\oplus$ V<sub>2,16</sub> $\oplus$ K<sub>2,16</sub> $\oplus$ (P<sub>2</sub> $\oplus$ K<sub>1,2</sub>) $\oplus$ (P<sub>3</sub> $\oplus$ K<sub>1,3</sub>) $\oplus$ (P<sub>4</sub> $\oplus$ K<sub>1,4</sub>) <br />
$\quad$ $\quad$ $\oplus$ V<sub>3,1</sub> $\oplus$ V<sub>3,4</sub> $\oplus$ V<sub>2,16</sub> $\oplus$ K<sub>3,2</sub> $\oplus$ V<sub>3,13</sub> $\oplus$ V<sub>3,16</sub> $\oplus$ V<sub>2,15</sub> $\oplus$ K<sub>3,14</sub> = 0
  
Simplifying the equation.

$\quad$ V<sub>3,1</sub> $\oplus$ V<sub>3,4</sub> $\oplus$ V<sub>3,13</sub> $\oplus$ V<sub>3,16</sub> $\oplus$ P<sub>2</sub> $\oplus$ P<sub>3</sub> <br />
$\quad$ $\quad$ $\oplus$ P<sub>4</sub> $\oplus$ K<sub>1,2</sub> $\oplus$ K<sub>1,3</sub> $\oplus$ K<sub>1,4</sub> $\oplus$ K<sub>2,16</sub> $\oplus$  K<sub>3,2</sub> $\oplus$ K<sub>3,14</sub> = 0
  
Note that  

$\quad$ $(a)$ U<sub>4,7</sub> = K<sub>4,7</sub> $\oplus$ V<sub>3,1</sub> $\Rightarrow$ V<sub>3,1</sub> = U<sub>4,7</sub> $\oplus$ K<sub>4,7</sub>  

$\quad$ $(b)$ U<sub>4,5</sub> = K<sub>4,5</sub> $\oplus$ V<sub>3,4</sub> $\Rightarrow$ V<sub>3,4</sub> = U<sub>4,5</sub> $\oplus$ K<sub>4,5</sub>  

$\quad$ $(c)$ U<sub>4,6</sub> = K<sub>4,6</sub> $\oplus$ V<sub>3,13</sub> $\Rightarrow$  V<sub>3,13</sub> = U<sub>4,6</sub> $\oplus$ K<sub>4,6</sub>  

$\quad$ $(d)$ U<sub>4,2</sub> = K<sub>4,2</sub> $\oplus$ V<sub>3,16</sub> $\Rightarrow$ V<sub>3,16</sub> = U<sub>4,2</sub> $\oplus$ K<sub>4,2</sub>  
  
$\quad$ $\therefore$ U<sub>4,7</sub> $\oplus$ K<sub>4,7</sub> $\oplus$ U<sub>4,5</sub> $\oplus$ K<sub>4,5</sub> $\oplus$ U<sub>4,6</sub> $\oplus$ K<sub>4,6</sub> $\oplus$ U<sub>4,2</sub> $\oplus$ K<sub>4,2</sub> $\oplus$ P<sub>2</sub> <br />
$\quad$ $\quad$ $\oplus$ P<sub>3</sub> $\oplus$ P<sub>4</sub> $\oplus$ K<sub>1,2</sub> $\oplus$ K<sub>1,3</sub> $\oplus$ K<sub>1,4</sub> $\oplus$ K<sub>2,16</sub> $\oplus$  K<sub>3,2</sub> $\oplus$ K<sub>3,14</sub> = 0  
  
Let $\sum_{k}$ = K<sub>4,7</sub> $\oplus$ K<sub>4,5</sub> $\oplus$ K<sub>4,6</sub> $\oplus$ K<sub>4,2</sub> $\oplus$ K<sub>1,2</sub> $\oplus$ K<sub>1,3</sub> $\oplus$ K<sub>1,4</sub> $\oplus$ K<sub>2,16</sub> $\oplus$  K<sub>3,2</sub> $\oplus$ K<sub>3,14</sub>  
  
We get the following equation. 

$\quad$ U<sub>4,7</sub> $\oplus$  U<sub>4,5</sub> $\oplus$ U<sub>4,6</sub> $\oplus$  U<sub>4,2</sub> $\oplus$ P<sub>2</sub> $\oplus$ P<sub>3</sub> $\oplus$ P<sub>4</sub> $\oplus$ $\sum_{k}$ = 0
  
Calculating probabilty. 

$\quad$ $\frac{1}{2} + 2^3(\frac{12}{16}-\frac{1}{2})(\frac{12}{16}-\frac{1}{2})(\frac{4}{16}-\frac{1}{2})(\frac{4}{16}-\frac{1}{2}) = \frac{17}{32}$  
  
Depending on whether  $\sum_{k}$ is 0 or 1, the probability is $\frac{17}{32}$ or $\frac{15}{32}$ (with bias of magnitude $\frac{1}{32}$)

### 2.1.1 Final Equation:

$\quad$ U<sub>4,2</sub> $\oplus$ U<sub>4,5</sub> $\oplus$ U<sub>4,6</sub> $\oplus$ U<sub>4,7</sub> $\oplus$ P<sub>2</sub> $\oplus$ P<sub>3</sub> $\oplus$ P<sub>4</sub> $\oplus$ $\sum_{k}$ = 0

A visualisation of the derivation of the equation can be found [here](./images/linear%20equation%201%20(msb).png).

### 2.1.2 Initialising the Substitution Permutation Network

In [5]:
spn = SPN()

For the purposes of this demonstration, we will be carrying out a known plaintext attack. In assuming the role of the attacker, we will use a pre-generated list of 10000 plaintexts and corresponding ciphertexts, which can be found in data.py. The code used to generate the plaintexts and ciphertexts can be found at the bottom of data.py.

### 2.1.3 Mounting the attack through Linear Approximation

We will now try to find the target partial subkey bits from the last round key associated with the substition boxes in the last round of the network.  

As per the equations derived, the bits at positions 2, 5, 6, and 7 of the input to the last substitution round and the bits at positions 2, 3, and 4 of the plaintext will be used. As the target partial subkey represent the 8 most significant bits (bits at indexes 0-7), the mode will be set to 'first'. As the keyspace for the target partial subkey is of size 2<sup>8</sup>, 256 different subkeys will be tested against the equation. If the equation holds, the count associated with that target partial subkey will be incremented. A larger count will denote that the target partial subkey has a higher probability of matching part of the last round key.

In [6]:
keyspace = test_partial_key_eqn(u_array=[2,5,6,7], p_array= [2,3,4], mode = "first")

### 2.1.4 Calculating the bias associated with each subkey

As there are 10000 plaintext-ciphertext pairs, the deviation bias for each subkey is given as follows:

$\quad$ $|\ bias\ | = |\ count - 5000\ |\ /\ 10000$

We take the top 10% of the keyspace based on their bias value.

In [8]:
linear_attack_bias_df = calculate_bias(keyspace, 25)
visualise_top_25_subkeys(linear_attack_bias_df)

subkey,bias
164,0.033000
173,0.028000
52,0.026200
84,0.026100
162,0.025600
244,0.025400
100,0.025000
196,0.024700
24,0.024400
1,0.024200


It can be noted that the largest bias has a value of 0.033, which is close to the theoretical value of $\frac{1}{32}$, or 0.03125.  

We save the top 10% subkeys in an array.

In [9]:
arr = linear_attack_bias_df["subkey"].to_numpy()
first_8bit_array = arr.tolist()
print(first_8bit_array)

[164, 173, 52, 84, 162, 244, 100, 196, 24, 1, 49, 61, 160, 40, 94, 56, 17, 23, 111, 33, 116, 212, 199, 4, 88]


### 2.2 Subkey representing the 8 least significant bits

We repeat the same steps as above for another set of target partial subkeys that represent the 8 least significant bits of the last round key.

#### We use the following approximations of the S-box:  
$\quad$ S<sub>14</sub>: X<sub>1</sub> $\oplus$ X<sub>2</sub> $\oplus$ X<sub>3</sub> $\oplus$ X<sub>4</sub> = Y<sub>1</sub>  

$\quad$ S<sub>22</sub>: X<sub>2</sub> = Y<sub>1</sub> $\oplus$ Y<sub>4</sub>  

$\quad$ S<sub>33</sub>: X<sub>2</sub> $\oplus$ X<sub>4</sub> = Y<sub>4</sub>

### 2.2.1 Final Equation:

$\quad$ U<sub>4,11</sub> $\oplus$ P<sub>13</sub> $\oplus$ P<sub>14</sub> $\oplus$ P<sub>15</sub> $\oplus$ P<sub>16</sub> $\oplus$ $\sum_{k}$ = 0

A visualisation of the derivation of the equation can be found [here](./images/linear%20equation%202%20(lsb).png).

### 2.2.2 Mounting the attack through Linear Approximation

In [10]:
keyspace2 = test_partial_key_eqn(u_array=[11], p_array= [13,14,15,16], mode = "last")

### 2.2.3 Calculating the bias associated with each subkey

In [11]:
linear_attack_bias_df = calculate_bias(keyspace2, 25)
visualise_top_25_subkeys(linear_attack_bias_df)

subkey,bias
89,0.030800
95,0.030800
80,0.030800
81,0.030800
82,0.030800
83,0.030800
84,0.030800
85,0.030800
86,0.030800
87,0.030800


We save the top 10% subkeys in an array.

In [12]:
arr = linear_attack_bias_df["subkey"].to_numpy()
last_8bit_array = arr.tolist()
print(last_8bit_array)

[89, 95, 80, 81, 82, 83, 84, 85, 86, 87, 88, 90, 91, 92, 93, 94, 128, 143, 136, 129, 142, 141, 140, 139, 137]


## 3 Combining the subkeys

We will now combine the subkeys that represent the 8 most significant bits and the 8 least significant bits to form 16 bit key. Each 16 bit key formed will be used to encrypt the plaintext, which will then be checked against the original plaintext. If they match, the key is said to be successfully obtained.

In [13]:
test_keys = generate_test_keys(first_8bit_array, last_8bit_array)
test_encryption(known_plaintext[:100], known_ciphertext[:100], test_keys)

Key Found: 42069


42069

Since our SPN uses the master key for all round keys, the above test can be carried out to verify that the key has been successfully obtained.  

The found key (42069) is the key that was used for encrypting all of the known plaintexts, thus impling that the attack was successful.

## 4 Limitations of the Project and Possible Extensions

### 4.1 Different round keys

The current implementation of the SPN uses the same keys for each round, which is the original key input by the user. As such, it is currently simple to verify the correctness of the attack and the derived key. If different round keys are used, the attacker will have to work the entire way up the SPN before doing the verification, deriving the different round keys for each round. The correctness is harder to verify in this scenario as the correctness of the round key for round i is based on the correctness of the round key for round i+1.  

Instead of having the identical round keys, different round keys could be generated through key schedules such as the DES algorithm.

### 4.2 Restricted target partial subkeys

The current implementation limits the subkeys to either the first 8 bits or the last 8 bits. As a result, the derived linear equations for use in the linear approximation must correspond to either the first 8 bits or the last 8 bits. This limits the number of equations that can be derived.  

The project could be improved by increasing the flexibility of possible subkey configurations, such as the bits from positions 5-12, or even from positions 1-4 and 13-16, or even just 4 bits for each target partial subkey.

### 4.3 Length of plaintext

The current implementation is limited to 16 bit inputs. This limits the depth of our analysis.  

To allow longer plaintexts, we can consider sufficiently strongs modes of encryption such as CTR and CBC.

### 4.4 Repeated use of Block Ciphers

The current implementation uses the same substitution boxes and permutation boxes for each round. As a result, there is less randomness in the encryption process.  

As an improvement, different substition boxes could be used. This strengthens the encryption as the linear cryptanalysis will be more time-consuming as each substition box requires its own linear approximation table. This will result in more time required to derive the linear equations for the attack.